In [ ]:
import sklearn.preprocessing
import tensorboard
import tensorflow as tf
import numpy as np
import keras.backend

from packaging import version
from datetime import datetime
from linkmethods import get_target, get_backlink_views
from keras.layers import *

In [ ]:
print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."

In [ ]:
tensorboard.__version__

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
article = "Meisen"

In [ ]:
df_main = get_target(article)

In [ ]:
bl_views = get_backlink_views(article, get_target(article))

In [ ]:
roundoff = 10

In [ ]:
roundeddata = []
for entry in bl_views:
    roundeddata.append((entry[:]//roundoff*roundoff)[-250:-1])
np.shape(roundeddata)

In [ ]:
roundeddata = np.array(roundeddata)

In [ ]:
data = np.array(bl_views)[:,-250:-1]
print(data)
data.shape

In [ ]:
target = np.array(df_main.views, dtype=float)[-250:-1]
target = np.array([np.copy(target) for _ in range((data.shape[0]))])
print(target)
target.shape

In [ ]:
roundedtarget = target//roundoff*roundoff
np.shape(roundedtarget)

In [ ]:
roundedtarget = np.array(roundedtarget)

In [ ]:
def round_activation(x):
  return tf.keras.backend.round(x/10)*10

In [ ]:
a = tf.constant([1,2,3])
type(a)

In [ ]:
def smape(target, forecast):
    #target = tf.constant(target)
    #forecast = tf.constant(forecast)
    denominator = tf.abs(target) + tf.abs(forecast)
    flag = denominator == 0.
    if denominator == 0 :
        smape = tf.multiply(2, tf.divide(tf.multiply(tf.abs(target - forecast), (1 - flag)), 1))
    else :
        smape = tf.multiply(2, tf.divide(tf.multiply(tf.abs(target - forecast), (1 - flag)), 1))

    return smape

In [ ]:
# Define the model.
model = keras.Sequential()

# 1 dense hidden layer with 250 nodes ~90+ acc
model.add(Input(shape=249)) # Input tensor
model.add(Dense(units=249)) # hidden layer 1,
model.add(Activation(activation="sigmoid"))

model.compile(
    optimizer='Adam',
    loss=smape,
    metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model.png")

In [ ]:
roundeddata = (roundeddata-roundeddata.mean())/roundeddata.std()
roundedtarget = (roundedtarget-roundedtarget.mean())/roundedtarget.std()

In [ ]:
roundeddata

In [ ]:
predata = sklearn.preprocessing.normalize(data,norm="l1")
pretarget = sklearn.preprocessing.normalize(target,norm="l1")
predata

In [ ]:
from sklearn.model_selection import train_test_split

# Separate the test data
x, x_test, y, y_test = train_test_split(data, target, test_size=0.15, shuffle=True)

# Split the remaining data to train and validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.15, shuffle=True)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
# Train the model.
early_stop = keras.callbacks.EarlyStopping(monitor="accuracy", patience=50, restore_best_weights=True)
history = model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=1500,
    validation_data=(x_val, y_val))

In [ ]:
from matplotlib import pyplot as plt
def display_learning_curves(history):
    fig, axs = plt.subplots(2, 2, figsize=(19, 11))

    axs[0,0].plot(history.history["loss"])
    axs[0,0].legend(["train", "test"], loc="upper right")
    axs[0,0].set_xlabel("Epochs")
    axs[0,0].set_ylabel("Loss")

    axs[0,1].plot(history.history["accuracy"])
    axs[0,1].legend(["train", "test"], loc="upper right")
    axs[0,1].set_xlabel("Epochs")
    axs[0,1].set_ylabel("Accuracy")

    axs[1,0].plot(history.history["val_loss"])
    axs[1,0].legend(["train", "test"], loc="upper right")
    axs[1,0].set_xlabel("Epochs")
    axs[1,0].set_ylabel("val_loss")

    axs[1,1].plot(history.history["val_accuracy"])
    axs[1,1].legend(["train", "test"], loc="upper right")
    axs[1,1].set_xlabel("Epochs")
    axs[1,1].set_ylabel("val_Accuracy")
    plt.show()

In [ ]:
display_learning_curves(history)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit --host localhost --port 8088

In [ ]:
keras.utils.plot_model(model, "my_first_model.png")

In [ ]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)

In [ ]:
weights = model.get_weights()
weights

In [ ]:
len(weights)